In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
import torch
import torchio as tio
import h5py
from ipywidgets import interact
import matplotlib.pyplot as plt
from pathlib import Path

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

In [ ]:
dataset_path = Path('X:\\Datasets\\2021 TC2See fMRI Data\\')
project_path = dataset_path / 'project'
derivatives_path = dataset_path / 'derivatives'

ssd_dataset_path = Path('C:\\Datasets\\2021 TC2See fMRI Data\\')
ssd_derivatives_path = ssd_dataset_path / 'derivatives'

In [ ]:
# Initial creation of h5 dataset for TC2See

import torchio as tio
import json
import pandas as pd
from tqdm.notebook import tqdm

break

with h5py.File(ssd_derivatives_path / 'TC2See2021.hdf5', 'w') as f:
    for subject_name in ('sub-01', 'sub-02'):
        subject = f.create_group(subject_name)
        derivatives_subject_path = derivatives_path / 'fmriprep' / subject_name
        project_subject_path = project_path / subject_name
        
        t1_file_path = derivatives_subject_path / 'anat' / f'{subject_name}_desc-preproc_T1w.nii.gz'
        t1_image = tio.ScalarImage(t1_file_path)
        t1_image.load()
    
        subject['anatomy/T1w'] = t1_image.data
        for k, v in t1_image.items():
            if k == 'data':
                continue
            subject['anatomy/T1w'].attrs[k] = v
            
        for i in tqdm(range(6)):
            run_id = i + 1
            
            bold_file_name = f'{subject_name}_task-bird_run-{run_id}_space-T1w_desc-preproc_bold'
            bold_file_path = derivatives_subject_path / 'func' / f'{bold_file_name}.nii.gz'
            
            bold_metadata_path = derivatives_subject_path / 'func' / f'{bold_file_name}.json'
            with open(bold_metadata_path) as metadata_file:
                bold_metadata = json.load(metadata_file)
            
            events_file_name = f'{subject_name}_task-bird_run-{run_id}_events.tsv'
            events_path = project_subject_path / 'func' / events_file_name
            with open(events_path) as events_file:
                events_text = events_file.read()
                
            bold_image = tio.ScalarImage(bold_file_path, events=events_text, **bold_metadata)
            bold_image.load()
            
            run_group = subject.create_group(f'bird_task/runs/run_{run_id}')
            run_group['data'] = bold_image.data
            for k, v in bold_image.items():
                if k == 'data':
                    continue
                run_group.attrs[k] = v
                

In [ ]:
# Add additional normalization data

def require_dataset(group, name, data):
    group.require_dataset(name, shape=data.shape, dtype=data.dtype)
    group[name][:] = data


with h5py.File(ssd_derivatives_path / 'TC2See2021.hdf5', 'a') as f:
    for subject_name, subject in f.items():
        for run in tqdm(subject['bird_task/runs'].values()):
            data = run['data'][:]
            
            H, W, D, T = data.shape
            data = torch.from_numpy(data).cuda()
                
            voxel_mean = data.mean(dim=3).cpu().numpy()
            voxel_std = data.std(dim=3).cpu().numpy()

            volume_mean = data.mean(dim=(0, 1, 2)).cpu().numpy()
            volume_std = data.mean(dim=(0, 1, 2)).cpu().numpy()

            run_mean = data.mean().cpu().item()
            run_std = data.std().cpu().item()

            require_dataset(run, 'voxel_mean', voxel_mean)
            require_dataset(run, 'voxel_std', voxel_std)
            require_dataset(run, 'volume_mean', volume_mean)
            require_dataset(run, 'volume_std', volume_std)

            run.attrs['run_mean'] = run_mean
            run.attrs['run_std'] = run_std

            B = data[..., None]

            A = torch.zeros_like(data)
            A[:, :, :, torch.arange(T)] = torch.arange(T).float().cuda()
            A = torch.stack([A, torch.ones_like(A)], dim=-1)

            X, _, _, _ = torch.linalg.lstsq(A, B)

            std = (A @ X - B).std(dim=3).squeeze().cpu().numpy()
            X = X.cpu().numpy()

            require_dataset(run, 'voxel_linear_trend', X)
            require_dataset(run, 'voxel_linear_trend_std', std)

            T = volume_mean.shape[0]
            B = torch.from_numpy(volume_mean[:, None])
            A = torch.arange(T).float()
            A = torch.stack([A, torch.ones_like(A)], dim=-1)

            X, _, _, _ = torch.linalg.lstsq(A, B)

            std = (A @ X - B).std()
            X = X.cpu().numpy()

            require_dataset(run, 'volume_linear_trend', X)
            require_dataset(run, 'volume_linear_trend_std', np.array([std]))

In [ ]:
# Load the h5 dataset

from research.data.tc2see_2021 import TC2See2021

features_name = 'ViT-B=32'
features_path = derivatives_path / f"{features_name}-features.hdf5"

dataset = TC2See2021(
    h5_path=ssd_derivatives_path / 'TC2See2021.hdf5', 
    subjects=['sub-01'],
    window=(3, 5),
    window_kernel=[1. / 2.] * 2,
    normalization='voxel_linear_trend',
    drop_out_of_window_events=True,
    features_path=features_path,
    feature_keys=['embedding'],
)

In [ ]:
dataset[0]['data'].shape

In [ ]:
# Visualize the events

@interact(i=(0, len(dataset)-1))
def show_event(i):
    event = dataset[i]
    
    if 'features' in event:
        for k, v in event['features'].items():
            print(k, v.shape, v.numel())
    
    #print(event['onset'], event['stimulus_id'], event['run_id'])
    data = event['data']
    #print(data.max(), data.min())
    #print(data.shape)
    
    print(event.keys())
    
    T, H, W, D = data.shape
    @interact(d=(0, D-1), t=(0, T-1))
    def show_volume(d, t):
        fig = plt.figure(figsize=(12, 12))
        x = data[t, :, :, d]
        plt.imshow(x, cmap='bwr', vmin=-3, vmax=3)
        plt.show()
        plt.close(fig)

In [ ]:
# Cache a preprocessing option

from research.data.tc2see_2021 import TC2See2021
from pathlib import Path
from tqdm.notebook import tqdm

def require_dataset(group, name, data):
    group.require_dataset(name, shape=data.shape, dtype=data.dtype)
    group[name][:] = data

cache_name = 'window-4-6'
subjects = ['sub-01', 'sub-02']
target_shape = (72, 88, 74)
preprocessing_params = dict(
    window=(2, 4),
    #window_kernel=[1. / 2.] * 2,
    normalization='voxel_linear_trend',
    drop_out_of_window_events=True,
    #features_path=features_path,
    #feature_keys=['embedding'],
)

with h5py.File(ssd_derivatives_path / 'TC2See2021-cached.hdf5', 'a') as f:
    for subject in subjects:
        print(subject)
        
        dataset = TC2See2021(
            h5_path=ssd_derivatives_path / 'TC2See2021.hdf5', 
            subjects=[subject],
            **preprocessing_params
        )
        N = len(dataset)

        sample_event = dataset[0]
        target_shape = sample_event['data'].shape

        group = f.require_group(f'{subject}/{cache_name}')
        data = group.require_dataset('data', shape=(N, *target_shape), dtype=float)
        for k, v in preprocessing_params.items():
            group.attrs[k] = str(v)

        group.attrs['affine'] = sample_event['affine']
        
        out_data = {
            k: [] for k in ['onset', 'duration', 'class_id', 'response', 
                            'response_time', 'same', 'tr', 'run_id', 'stimulus_id']
        }
        for i in tqdm(range(len(dataset))):
            event = dataset[i]
            for k, v in out_data.items():
                v.append(event[k])
            data[i] = event['data']
        for k, v in out_data.items():
            v = np.array(v)
            if k == 'stimulus_id':
                v = v.astype(np.dtype('S'))
            require_dataset(group, k, v)

In [ ]:
# Create correlation maps

from pathlib import Path
from tqdm.notebook import tqdm
from itertools import product
import time


def pearsonr_torch(X, Y):
    X = X.to(torch.float64)
    Y = Y.to(torch.float64)
    
    X = X - X.mean(dim=0, keepdim=True)
    Y = Y - Y.mean(dim=0, keepdim=True)
    
    X = X / torch.norm(X, dim=0, keepdim=True)
    Y = Y / torch.norm(Y, dim=0, keepdim=True)
    
    X_num_correlation_dims = len(X.shape) - 1
    Y_num_correlation_dims = len(Y.shape) - 1
    for i in range(Y_num_correlation_dims):
        X = X[..., None]
    for i in range(X_num_correlation_dims):
        Y = Y[:, None]
    
    return torch.einsum('b...i,b...i->...i', X, Y)


fmri_data = h5py.File(ssd_derivatives_path / 'TC2See2021-cached.hdf5', 'r')

cache_name = 'window-4-6'
run_features = {
    #'bigbigan-resnet50': ['z_mean'],
    'ViT-B=32': ['embedding', *(f'transformer.resblocks.{i}' for i in range(12))],
    #'biggan-128': ['z', 'y_embedding'],
    #'vqgan': ['vqgan-f16-1024-pre_quant'],
}

seed = 0
max_features = 512

with h5py.File(ssd_derivatives_path / 'feature-correlation-maps.hdf5', 'a') as f:
    for subject_name, subject in fmri_data.items():
        print(subject_name)

        if cache_name not in subject:
            continue
            print(f'{cache_name} not found for subject {subject_name}')
            
        cache = subject[cache_name]
        affine = cache.attrs['affine']
        group = f.require_group(f'{subject_name}/{cache_name}')
        group.attrs['affine'] = affine
        X = cache['data']

        for model_name, feature_names in run_features.items():
            print(model_name)
            model_features = h5py.File(derivatives_path / f'{model_name}-features.hdf5', 'r')

            for feature_name in feature_names:
                print(feature_name)

                stimulus_ids = subject[f'{cache_name}/stimulus_id'][:]
                stimulus_ids = [s.decode('utf-8') for s in stimulus_ids]
                Y = np.stack([model_features[f'{stimulus_id}/{feature_name}'][:]
                              for stimulus_id in stimulus_ids])
                Y = torch.from_numpy(Y).cuda()
                Y = Y.flatten(start_dim=1)

                if Y.shape[1] > max_features:
                    np.random.seed(seed)
                    choice = np.random.choice(max_features, size=max_features)
                    Y = Y[:, choice]

                correlation_map_shape = X.shape[1:] + Y.shape[1:]
                correlation_map = f.require_dataset(f'{subject_name}/{cache_name}/{model_name}/{feature_name}', correlation_map_shape, dtype=np.float32)

                load_time = 0
                compute_time = 0
                store_time = 0
                for i in tqdm(range(X.shape[1])):

                    t = time.time()
                    X_slice = torch.from_numpy(X[:, i]).cuda()
                    load_time += time.time() - t
                    

                    t = time.time()
                    r = torch.stack([
                        pearsonr_torch(X_slice[:, j], Y).cpu()
                        for j in range(X_slice.shape[1])
                    ])
                    compute_time += time.time() - t

                    t = time.time()
                    correlation_map[i] = r
                    store_time += time.time() - t

                    if i % 25 == 0:
                        n = (i + 1)
                        print(f'load_time={load_time / n}, compute_time={compute_time / n}, store_time={store_time / n}')

In [ ]:
# Save correlation maps
from pathlib import Path
import torchio as tio
from functools import partial
import nibabel as nib

def require_dataset(group, name, data):
    group.require_dataset(name, shape=data.shape, dtype=data.dtype)
    group[name][:] = data


def mean_top_k(data, k):
    data = np.abs(data)
    data = np.sort(data)
    data = data[..., -k:].mean(axis=-1)
    return data

def max_feature(data):
    data = np.abs(data)
    data = np.max(data, axis=-1)
    return data

selection_modes = {
    #'max': max_feature,
    'mean-top-5': partial(mean_top_k, k=5),
    #'mean-top-10': partial(mean_top_k, k=10),
}
models = ['ViT-B=32']
cache_names = ['window-4-6']

out_path = derivatives_path / 'correlation_maps'
correlation_maps = h5py.File(ssd_derivatives_path / 'feature-correlation-maps.hdf5', 'r')

with h5py.File(ssd_derivatives_path / 'feature-selection-maps.hdf5', 'a') as f:
    for subject_name, subject in correlation_maps.items():
        subject_out_path = out_path / subject_name
        subject_out_path.mkdir(parents=True, exist_ok=True)
        
        for cache_name, cache in subject.items():
            if cache_names and cache_name not in cache_names:
                continue
            affine = cache.attrs['affine']
            for model_name, model in cache.items():
                if model_name not in models:
                    continue

                for feature_name, feature_correlation_map in model.items():
                    for selection_mode, selection_func in selection_modes.items():
                        keys = (subject_name, cache_name, model_name, feature_name, selection_mode)
                        print(*keys)

                        save_file_name = f'{"__".join(keys)}.nii.gz'

                        data = feature_correlation_map[:]
                        data = selection_func(data)
                        sorted_indices_flat = np.argsort(data, axis=None)

                        T, H, W, D = data.shape
                        grid = np.zeros(shape=(4, T, H, W, D), dtype=int)
                        grid[0] = np.arange(T)[:, None, None, None]
                        grid[1] = np.arange(H)[None, :, None, None]
                        grid[2] = np.arange(W)[None, None, :, None]
                        grid[3] = np.arange(D)[None, None, None, :]
                        grid_flat = grid.reshape(4, T * H * W * D)
                        sorted_indices = grid_flat[:, sorted_indices_flat]
                        
                        image = nib.Nifti1Image(torch.tensor(data).permute(1, 2, 3, 0).numpy(), affine)
                        nib.save(image, subject_out_path / save_file_name)
                        #image = tio.ScalarImage(tensor=torch.tensor(data), affine=affine)
                        #image.save(subject_out_path / save_file_name)

                        group = f.require_group('/'.join(keys))
                        require_dataset(group, 'scores', data)
                        require_dataset(group, 'sorted_indices_flat', sorted_indices_flat)
                        require_dataset(group, 'sorted_indices', sorted_indices)

correlation_maps.close()

In [ ]:
# stack feature selection map
from pathlib import Path
import torchio as tio
from functools import partial
import nibabel as nib


cache_name = 'window-4-6'
model_name = 'ViT-B=32'
feature_names = [*(f'transformer.resblocks.{i}' for i in range(12)), 'embedding']
selection_mode = 'mean-top-5'
stack_name = 'depth-sequence-time-4'
channel = 0

correlation_maps = h5py.File(ssd_derivatives_path / 'feature-correlation-maps.hdf5', 'r')
out_path = derivatives_path / 'correlation_maps'

with h5py.File(ssd_derivatives_path / 'feature-selection-maps.hdf5', 'a') as f:
    for subject_name, subject in correlation_maps.items():
        subject_out_path = out_path / subject_name
        subject_out_path.mkdir(parents=True, exist_ok=True)
        
        affine = correlation_maps[subject_name][cache_name].attrs['affine']
        features = f[subject_name][cache_name][model_name]
        data = np.stack([
            features[feature_name][selection_mode]['scores'][channel]
            for feature_name in feature_names
        ])
        print(data.shape)
        
        keys = (subject_name, cache_name, model_name, selection_mode, stack_name,)
        print(*keys)

        save_file_name = f'{"__".join(keys)}.nii.gz'
        image = nib.Nifti1Image(torch.tensor(data).permute(1, 2, 3, 0).numpy(), affine)
        nib.save(image, subject_out_path / save_file_name)

correlation_maps.close()

In [ ]:
# Shuffle estimator

from pathlib import Path
from tqdm.notebook import tqdm
from itertools import product
import time


def pearsonr_torch(X, Y):
    X = X.to(torch.float64)
    Y = Y.to(torch.float64)
    
    X = X - X.mean(dim=0, keepdim=True)
    Y = Y - Y.mean(dim=0, keepdim=True)
    
    X = X / torch.norm(X, dim=0, keepdim=True)
    Y = Y / torch.norm(Y, dim=0, keepdim=True)
    
    X_num_correlation_dims = len(X.shape) - 1
    Y_num_correlation_dims = len(Y.shape) - 1
    for i in range(Y_num_correlation_dims):
        X = X[..., None]
    for i in range(X_num_correlation_dims):
        Y = Y[:, None]
    
    return torch.einsum('b...i,b...i->...i', X, Y)


def max_pearsonr_partition_estimator(X, Y, partitions, seed=0):
    N = X_slice.shape[0]
    
    measurements = []
    for j in tqdm(range(partitions)):
        np.random.seed(seed + j)

        indices = np.arange(N)
        np.random.shuffle(indices)
        indices1, indices2 = np.split(indices, 2)

        X1, Y1 = X[indices1], Y[indices1]
        X2, Y2 = X[indices2], Y[indices2]

        r1 = torch.abs(torch.stack([
            pearsonr_torch(X1[:, k], Y1).cpu()
            for k in range(X1.shape[1])
        ]))

        r2 = torch.abs(torch.stack([
            pearsonr_torch(X2[:, k], Y2).cpu()
            for k in range(X2.shape[1])
        ]))

        max_ids = r1.argmax(axis=-1)
        measurement = torch.gather(r2, -1, max_ids[..., None])
        measurements.append(measurement)
    
    r = torch.cat(measurements, dim=-1)
    return r


fmri_data = h5py.File(ssd_derivatives_path / 'TC2See2021-cached.hdf5', 'r')

cache_name = 'window-4-6'
run_features = {
    #'bigbigan-resnet50': ['z_mean'],
    'ViT-B=32': ['embedding'],#, *(f'transformer.resblocks.{i}' for i in (2, 5, 8, 11))],
    #'biggan-128': ['z', 'y_embedding'],
    #'vqgan': ['vqgan-f16-1024-pre_quant'],
}

seed = 0
max_features = 512
partitions = 50

with h5py.File(ssd_derivatives_path / 'feature-max-correlation-maps.hdf5', 'a') as f:
    for subject_name, subject in fmri_data.items():
        print(subject_name)

        if cache_name not in subject:
            continue
            print(f'{cache_name} not found for subject {subject_name}')
            
        cache = subject[cache_name]
        affine = cache.attrs['affine']
        group = f.require_group(f'{subject_name}/{cache_name}')
        group.attrs['affine'] = affine
        X = cache['data']

        for model_name, feature_names in run_features.items():
            print(model_name)
            model_features = h5py.File(derivatives_path / f'{model_name}-features.hdf5', 'r')

            for feature_name in feature_names:
                print(feature_name)

                stimulus_ids = subject[f'{cache_name}/stimulus_id'][:]
                stimulus_ids = [s.decode('utf-8') for s in stimulus_ids]
                Y = np.stack([model_features[f'{stimulus_id}/{feature_name}'][:]
                              for stimulus_id in stimulus_ids])
                Y = torch.from_numpy(Y).cuda()
                Y = Y.flatten(start_dim=1)

                if Y.shape[1] > max_features:
                    np.random.seed(seed)
                    choice = np.random.choice(max_features, size=max_features)
                    Y = Y[:, choice]

                correlation_map_shape = X.shape[1:] + Y.shape[1:-2] + (partitions,)
                print(correlation_map_shape)

                correlation_map = f.require_dataset(f'{subject_name}/{cache_name}/{model_name}/{feature_name}', correlation_map_shape, dtype=np.float32)

                load_time = 0
                compute_time = 0
                store_time = 0
                for i in tqdm(range(X.shape[1])):

                    t = time.time()
                    X_slice = torch.from_numpy(X[:, i]).cuda()
                    load_time += time.time() - t
                    
                    N = X_slice.shape[0]

                    t = time.time()
                    r = max_pearsonr_partition_estimator(X_slice, Y, partitions, seed=seed)
                    compute_time += time.time() - t

                    t = time.time()
                    correlation_map[i] = r
                    store_time += time.time() - t

                    if i % 25 == 0:
                        n = (i + 1)
                        print(f'load_time={load_time / n}, compute_time={compute_time / n}, store_time={store_time / n}')

In [ ]:
# Save correlation maps
from pathlib import Path
import torchio as tio
from functools import partial
import nibabel as nib

def require_dataset(group, name, data):
    group.require_dataset(name, shape=data.shape, dtype=data.dtype)
    group[name][:] = data


models = ['ViT-B=32']
cache_names = ['window-4-6']

out_path = derivatives_path / 'correlation_maps'
correlation_maps = h5py.File(ssd_derivatives_path / 'feature-max-correlation-maps.hdf5', 'r')

with h5py.File(ssd_derivatives_path / 'feature-selection-maps.hdf5', 'a') as f:
    for subject_name, subject in correlation_maps.items():
        subject_out_path = out_path / subject_name
        subject_out_path.mkdir(parents=True, exist_ok=True)
        
        for cache_name, cache in subject.items():
            if cache_names and cache_name not in cache_names:
                continue
            affine = cache.attrs['affine']
            for model_name, model in cache.items():
                if model_name not in models:
                    continue

                for feature_name, feature_correlation_map in model.items():
                    keys = (subject_name, cache_name, model_name, feature_name, 'partition-max')
                    print(*keys)

                    save_file_name = f'{"__".join(keys)}.nii.gz'

                    data = feature_correlation_map[:]
                    data = np.mean(data, axis=-1)
                    sorted_indices_flat = np.argsort(data, axis=None)

                    T, H, W, D = data.shape
                    grid = np.zeros(shape=(4, T, H, W, D), dtype=int)
                    grid[0] = np.arange(T)[:, None, None, None]
                    grid[1] = np.arange(H)[None, :, None, None]
                    grid[2] = np.arange(W)[None, None, :, None]
                    grid[3] = np.arange(D)[None, None, None, :]
                    grid_flat = grid.reshape(4, T * H * W * D)
                    sorted_indices = grid_flat[:, sorted_indices_flat]

                    image = nib.Nifti1Image(torch.tensor(data).permute(1, 2, 3, 0).numpy(), affine)
                    nib.save(image, subject_out_path / save_file_name)
                    #image = tio.ScalarImage(tensor=torch.tensor(data), affine=affine)
                    #image.save(subject_out_path / save_file_name)

                    group = f.require_group('/'.join(keys))
                    require_dataset(group, 'scores', data)
                    require_dataset(group, 'sorted_indices_flat', sorted_indices_flat)
                    require_dataset(group, 'sorted_indices', sorted_indices)

correlation_maps.close()

In [ ]:
r

In [ ]:
data.shape

In [ ]:
x = torch.randn(5, 100, 100)
max_ids = x.argmax(axis=-1)
torch.gather(x, -1, max_ids[..., None]).shape

In [ ]:
image = tio.ScalarImage(out_path / 'sub-01'/ 'sub-01__large-window__ViT-B=32__embedding__max.nii.gz')
image.load()
image

In [ ]:
import nibabel as nib
array_img = nib.Nifti1Image(image['data'].numpy(), image['affine'])

In [ ]:
correlation_maps = h5py.File(ssd_derivatives_path / 'feature-correlation-maps.hdf5', 'r')
#print(list(correlation_maps['sub-01/large-window/ViT-B=32/embedding'].keys()))

data = correlation_maps['sub-01/large-window/ViT-B=32/embedding'][:]
@interact(t=(0, data.shape[0]-1), d=(0, data.shape[3]-1), f=(0, data.shape[4]-1))
def show(t, d, f):
    plt.imshow(data[t, :, :, d, f], vmin=0, vmax=0.1)

In [ ]:
cached_data = h5py.File(ssd_derivatives_path / 'TC2See2021-cached.hdf5', 'r')

data = cached_data['sub-01/large-window/data'][:10]
@interact(b=(0, data.shape[0]-1), t=(0, data.shape[1]-1), d=(0, data.shape[4]-1))
def show(b, t, d):
    plt.imshow(data[b, t, :, :, d], vmin=-3, vmax=3.)

In [ ]:
pip install

In [ ]:
from vedo import Volume, dataurl
from vedo.applications import RayCastPlotter

embryo = Volume(dataurl+"embryo.slc").mode(1).c('jet') # change properties

plt = RayCastPlotter(embryo) # Plotter instance
plt.show(viewup="z", bg='black', bg2='blackboard', axes=7).close()

In [ ]:
f = h5py.File(ssd_derivatives_path / 'feature-selection-maps.hdf5', 'r')

subject = 'sub-02'
cache_name = 'window-4-6'
model_name = 'ViT-B=32'
feature_name = 'transformer.resblocks.0'
selection_mode = 'mean-top-5'

key = '/'.join([subject, cache_name, model_name, feature_name, selection_mode, 'scores'])

data = f[key][0]
(~np.isnan(data)).sum()

In [ ]:


def stuff(k, v):
    if not isinstance(v, h5py.Dataset):
        return
    if not v.name.endswith('scores'):
        return
    
    data = v[:]
    #data[np.isnan(data)] = 0.
    #v[:] = data
    print(v.name, v, np.isnan(v[:]).sum())
    
with h5py.File(ssd_derivatives_path / 'feature-selection-maps.hdf5', 'a') as f:
    f.visititems(stuff)